<a href="https://colab.research.google.com/github/r-u-5k/study-dev/blob/master/bigdata/ch2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형2 문제
제공된 데이터는 성인 인구조사 소득 데이터이다. 제공된 학습용 데이터(train.csv)를 이용하여 소득을 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(test.csv)에 적용하여 얻은 소득 예측 결과를 아래 [제출 형식] 에 따라 CSV 파일로 생성하는 코드를 제출하시오.

* 예측 결과는 ROC-AUC 평가 지표에 따라 평가함

* 평가는 '> 50K' 클래스를 positive(양성)으로 간주하여 계산함

* 성능이 우수한 예측 모델을 구축하기 위해서는 데이터 정제, Feature Engineering, 하이퍼 파라미터(hyper parameter) 최적화, 모델 비교 등이 필요할 수 있음. 다만, 과적합에 유의하여야 함.


### [제출 형식]

① CSV 파일명: result.csv (파일명에 디렉토리·폴더 지정불가)

② 예측 칼럼명: pred

③ 제출 칼럼 개수: pred 칼럼 1개

④ 평가용 데이터 개수와 예측 결과 데이터 개수 일치: 3,257개



### [데이터 설명]
- id : 고유의 id 값
- age : 나이
- workclass : 고용 형태
- fnlwgt : 사람 대표성을 나타내는 가중치 (final weight의 약자)
- education : 교육 수준
- education_num : 교육 수준 수치
- marital_status: 결혼 상태
- occupation : 업종
- relationship : 가족 관계
- race : 인종
- sex : 성별
- capital_gain : 양도 소득
- capital_loss : 양도 손실
- hours_per_week : 주당 근무 시간
- native_country : 국적
- income : 수익(예측해야 하는 값)
    - "> 50K" : 1
    - "<= 50K" : 0


📄 제출 CSV 파일 형식 예시 (result.csv)
~~~
pred
<= 50K
> 50K
> 50K
<= 50K
> 50K
...
<= 50K
~~~

# 1. 문제정의
- 레이블(타깃)은 연소득이 "> 50K"과 "<= 50K"으로 구분됨(컬럼명: income)

- 소득 예측값($50,000 보다 클 확률)을 csv 파일로 생성
- 평가 기준은 ROC-AUC로 평가
- 제출 파일은 예측값만 result.csv 파일로 생성해 제출(컬럼명: pred, 1개)

# 2. 라이브러리 및 데이터 불러오기

In [ ]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
train = pd.read_csv('https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p2/ch2/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p2/ch2/test.csv')

# 3. 탐색적 데이터 분석(EDA)

In [ ]:
# 데이터 샘플
train.head(3)

In [ ]:
# 데이터 크기
train.shape, test.shape

In [ ]:
# 자료형 (타입)
train.info()

In [ ]:
# 수치형 컬럼 통계값 확인(train)
train.describe()

In [ ]:
# 수치형 컬럼 통계값 확인(test)
test.describe()

In [ ]:
# 범주형 컬럼 통계값 확인 (train)
train.describe(include='O')

In [ ]:
# 범주형 컬럼 통계값 확인 (test)
test.describe(include='object')

In [ ]:
# 결측치 (train)
train.isnull().sum()

In [ ]:
# 결측치 (test)
test.isnull().sum()

In [ ]:
# label(target)별 개수 확인
train['income'].value_counts()

# 4. 데이터 전처리

1. 결측치 처리 (결측치가 있다면 필수)
2. 이상치 처리 (선택)
3. 인코딩 (범주형 데이터가 있다면 필수)
4. 스케일링 (선택)

## 1. 결측치 처리
- 범주형
- 수치형

### 결측치 삭제

In [ ]:
# 결측치가 있는 데이터(행) 전체 삭제 및 확인 dropna() / 기본값 axis=0
print("처리전:", train.shape)
df = train.dropna()
print("처리후:", df.shape)

In [ ]:
# 결측치가 있는 특정컬럼 데이터(행)삭제
df = train.dropna(subset=['native.country','workclass'])
df.isnull().sum()

In [ ]:
# 결측치가 있는 컬럼 삭제 dropna(axis=1)
print("처리전:", train.shape)
df = train.dropna(axis=1)
print("처리후:", df.shape)

In [ ]:
# 결측치가 많은 특정 컬럼 삭제 drop
print("처리전:", train.shape)
df = train.drop(['native.country','workclass'], axis=1)
print("처리후:", df.shape)

### 결측치 채우기 (범주형)

In [ ]:
# 최빈값
m = train['workclass'].mode()[0]
train['workclass'] = train['workclass'].fillna(m)

m = train['native.country'].mode()[0]
train['native.country'] = train['native.country'].fillna(m)

train.isnull().sum()

In [ ]:
# 결측값을 새로운 카테고리로 생성 X_train['occupation']
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

In [ ]:
# test데이터
test['workclass'] = test['workclass'].fillna(train['workclass'].mode()[0])
test['native.country']  = test['native.country'].fillna(train['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna("X")

### 결측치 채우기 (수치형)

In [ ]:
# age 컬럼 평균값으로 채우기
value = int(train['age'].mean())
print("평균값:", value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

In [ ]:
# 주당 근무시간 중앙값으로 채우기
value = train['hours.per.week'].median()
print("중앙값:", value)
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)

In [ ]:
train.isnull().sum()

## 이상치 처리

In [ ]:
train.describe()

In [ ]:
cond = train['age']<=0
train[cond]

In [ ]:
cond = test['age']<=0
test[cond]

In [ ]:
# age가 1이상인 데이터만 살림
print(train.shape)
cond = train['age']>0
train = train[cond]
print(train.shape)

전처리 주의사항
- 이상치, 결측치에 대해 test 데이터(행) 삭제 불가
    - 예) test 데이터 100개가 주어지고 100개로 평가를 하는데 임의로 10개를 삭제해버리면 채점을 할 수가 없음
    - train 데이터는 학습/훈련용 데이터임. 데이터가 많을 경우 임의로 소수 데이터(행) 삭제해도 무방함
- test, train 컬럼은 삭제/추가 가능. 단, train과 컬럼수와 명이 일치해야 함 (y(target) 제외)


## 인코딩

In [ ]:
y_train = train.pop("income")

### 원핫(one-hot) 인코딩

In [ ]:
train_oh = pd.get_dummies(train)
test_oh = pd.get_dummies(test)
print(train.shape, test.shape, train_oh.shape, test_oh.shape)

In [ ]:
print(train.shape, test.shape)

data = pd.concat([train, test], axis=0)
data_oh = pd.get_dummies(data)

train_oh = data_oh.iloc[:len(train)].copy()
test_oh = data_oh.iloc[len(train):].copy()

print(train_oh.shape, test_oh.shape)

### 레이블(label) 인코딩

In [ ]:
# cols = train.select_dtypes(include='object').columns #방법1
# cols = train.columns[train.dtypes == object] #방법2
cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex','native.country']
cols

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

train.head()

## 스케일링

In [ ]:
# 수치형 컬럼명
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

In [ ]:
# 데이터를 매번 새롭게 불러오기 위해 함수로 제작 함
def get_data():
    train_copy = train.copy()
    test_copy = test.copy()
    return train_copy, test_copy

In [ ]:
# 민-맥스 스케일링 MinMaxScaler (모든 값이 0과 1사이)
train_copy, test_copy = get_data()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

display(train_copy[cols].head(2))
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head(2))

In [ ]:
# 표준화 StandardScaler (Z-score 정규화, 평균이 0 표준편차가 1인 표준 정규분포로 변경)
train_copy, test_copy = get_data()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
display(train_copy[cols].head(2))
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head(2))

In [ ]:
# 로버스트 스케일링 : 중앙값과 사분위 값 활용, 이상치 영향 최소화 장점
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
display(train[cols].head(2))

## target(label)

In [ ]:
y_train.value_counts()

In [ ]:
# target = y_train.map({'<=50K':0, '>50K':1})
# target.value_counts()

In [ ]:
# target = y_train.replace('<=50K', 0).replace('>50K', 1)
# target.value_counts()

# 5. 검증 데이터 분할

In [ ]:
# 데이터 크기
train.shape, y_train.shape, test.shape

In [ ]:
# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train,
                                            y_train,
                                            test_size=0.2,
                                            random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

# 6. 머신러닝 학습 및 평가

### 추천 머신러닝 모델
1. 랜덤포레스트
2. lightGBM

### 머신러닝 학습 및 예측 방법 (3가지)
- 모델 불러오기
- 학습 fit(X, y)
- 예측 predict(test)
    - 또는 predict_proba() -> 평가지표가 roc-auc 때 주로 사용


In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred=rf.predict_proba(X_val) # 각 레이블에 속할 확률 값 반환

In [ ]:
# 예측 결과 확인
pred[:10]
# array([[1.  , 0.  ],
#        [1.  , 0.  ],
#        [0.9 , 0.1 ],
#        [0.63, 0.37],
#        [1.  , 0.  ],
#        [0.99, 0.01],
#        [0.98, 0.02],
#        [0.94, 0.06],
#        [0.12, 0.88],
#        [0.88, 0.12]])

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.9 , 0.1 ],
       [0.63, 0.37],
       [1.  , 0.  ],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.88, 0.12]])

In [ ]:
# 클래스 확인
print(rf.classes_)

['<=50K' '>50K']


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

roc_auc: 0.9173623004487484


In [ ]:
# predict
pred = rf.predict(X_val) # 예측된 각 레이블 반환
pred[:10]

array(['<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '>50K', '<=50K'], dtype=object)

In [ ]:
# 정확도 (accuracy)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

accuracy_score: 0.8694761986009213


In [ ]:
# f1
from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:', f1)

f1_score: 0.6926476496584973


In [ ]:
# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_tr, y_tr)
pred = lgbmc.predict_proba(X_val)

roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

pred = lgbmc.predict(X_val)
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:', f1)

roc_auc: 0.9279535666686397
accuracy_score: 0.8771540692714553
f1_score: 0.7158642462509865


# 7. 예측 및 결과 파일 생성

In [ ]:
pred = lgbmc.predict_proba(test)
pred

array([[0.89924007, 0.10075993],
       [0.97622077, 0.02377923],
       [0.9853122 , 0.0146878 ],
       ...,
       [0.93528667, 0.06471333],
       [0.98996872, 0.01003128],
       [0.96723292, 0.03276708]])

In [ ]:
lgbmc.classes_

array(['<=50K', '>50K'], dtype=object)

In [ ]:
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

In [ ]:
pd.read_csv("result.csv")

,pred
0,0.100760
1,0.023779
2,0.014688
3,0.798048
4,0.026542
...,...
3252,0.009613
3253,0.321992
3254,0.064713
3255,0.010031


### [심화] roc_auc 일때, predict vs predict_proba

In [ ]:
target = y_train.map({'<=50K':0, '>50K':1})
train.shape, target.shape

((29301, 15), (29301,))

In [ ]:
# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  target,
                                                  test_size=0.2,
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23440, 15), (5861, 15), (23440,), (5861,))

In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_val)

roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

roc_auc: 0.9173623004487484


In [ ]:
pred=rf.predict(X_val)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
roc_auc = roc_auc_score(y_val, pred)
print('roc_auc:', roc_auc)

roc_auc: 0.7861106214635613
